# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9, max_tokens=500)

In [2]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [3]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "性能卓越的GPU"
    }))



1. Elegant Edge Tech Corp 
2. Generous Graphics Group 
3. Magnificent Machine Limited 
4. Optimus Performance Ltd 
5. Prime Precision Ltd 
6. Savvy Systems Limited 
7. Superior Solutions Inc 
8. Expert Electronics Ltd 
9. Power Processing Ltd 
10. Innovative Imaging Inc


In [4]:
chain.verbose = True

In [5]:
chain.verbose

True

In [6]:
print(chain.run({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.


1.璀璨显卡公司（Brilliant Graphic Cards Company）
2.火线显卡有限公司（Fiery Graphics Cards Ltd.）
3.超级GPU有限公司（Super GPU Ltd.）
4.科技图形有限公司（Tech Graphics Ltd.）
5.旗舰显卡公司（Flagship Graphics Company）
6.3D显卡有限公司（3D Graphics Ltd.）
7.显卡精英公司（Graphics Elite Company）
8.GPU之家有限公司（GPU Home Ltd.）
9.4K显卡有限公司（4K Graphics Ltd.）
10.高性能显卡有限公司（High Performance Graphics Ltd.）


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [21]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

print(synopsis_chain.run({'title':'三体人不是无法战胜的'}))



《三体人不是无法战胜的》是一出关于勇气和希望的戏剧。故事发生在一个遭受战争破坏的世界，一群勇敢的异类以及他们的英雄般的行动，最终将导致变革。三体人是一种异类，他们被迫在这个破碎的世界中求生，但他们也拥有着不可思议的力量，可以让他们超越所有的困难。只要他们拥有勇气与希望，就没有什么是无法战胜的。


In [12]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [13]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [10]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


《三体人不是无法战胜的》讲述了一个不同族群的人之间的战斗。三体人是由一个宇宙中的三个原始文明组成的人类，他们各自拥有不同的特性，但却有着共同的目标：统一宇宙，成为宇宙中唯一的文明。在一场空前绝后的战斗中，三体人发现自己与邪恶的文明正在越来越靠近。三体人面临着以往从未想象过的危险，必须采取行动来拯救他们的宇宙。最终，他们发现，只有勇敢和坚定的信念才能战胜邪恶。


《三体人不是无法战胜的》是一部令人振奋的史诗剧，它讲述了一个充满勇气和坚定信念的故事。三体人是由三个原始文明组成的人类，他们有着不同的特性，但他们有着共同的目标：统一宇宙。这部剧让观众感受到一种强烈的力量，从而激发他们去追求自己的梦想。影片中的战斗场面恢弘壮观，让观众陷入一种紧张而又令人兴奋的氛围。剧中的人物形象生动有趣，他们的表现更是令人难以忘怀。总之，《三体人不是无法战胜的》是一部令人振奋的史诗剧，是一部令人兴奋而又感动的影片，值得一看。

> Finished chain.


In [14]:
review = overall_chain.run("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》描述了一群勇敢的英雄们再次出击，冒险去拯救他们的家园——一个遭受绝地武士攻击的银河系。在一段长期的旅程中，他们面临许多严峻的挑战，但他们仍然决心拯救他们的家园免遭毁灭，因此他们必须坚持不懈，抵抗绝地武士的攻击。在他们的行动中，他们发现一个惊人的真相，他们必须将它发掘出来，以扭转战局。最终，英雄们能否夺回家园，并终结这场长期的抗争？这是本剧所要探讨的问题。


《星球大战第九季》是一部极具想象力、充满动作的史诗级大片。这部剧拥有令人兴奋的剧情，让观众紧张不已，期待着看看主角们能否夺回家园并终结这场长期的抗争。在他们的行动中，他们发现一个惊人的真相，令他们深感震惊。剧中的英雄们拥有一种勇敢、坚定不移的品质，令人赞叹不已，而他们所面临的挑战也极具挑战性，这样的画面令观众更加兴奋。总之，《星球大战第九季》是一部出色的史诗级大片，可以让观众在观剧的同时，也深刻体会到其中所探讨的重大问题。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [15]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [16]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [17]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [18]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


三体人不是无法战胜的是一部关于新中国在二十一世纪对抗外星人的现代战争戏剧。故事发生在2050年的新中国，当一群外星人试图控制新中国时，一群勇敢的中国士兵和智能机器人联合起来抵抗外星人的侵略。在双方的较量中，新中国发现了外星人所控制力量的弱点，并计划利用这一弱点击败外星人。政府和民间势力一起为此而战，最终新中国在一场壮观的太空大战中取得胜利，宣告外星人不是无法战胜的。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n三体人不是无法战胜的是一部关于新中国在二十一世纪对抗外星人的现代战争戏剧。故事发生在2050年的新中国，当一群外星人试图控制新中国时，一群勇敢的中国士兵和智能机器人联合起来抵抗外星人的侵略。在双方的较量中，新中国发现了外星人所控制力量的弱点，并计划利用这一弱点击败外星人。政府和民间势力一起为此而战，最终新中国在一场壮观的太空大战中取得胜利，宣告外星人不是无法战胜的。',
 'review': '\n\n《三体人不是无法战胜的》是一部激动人心的现代战争戏剧，讲述了新中国在对抗外星人的过程中取得胜利的故事。该剧充分利用了科幻元素，给观众带来了一场精彩的太空大战，令人惊叹。剧中人物的性格刻画深刻，勇敢的中国士兵和智能机器人齐心协力，让观众深刻体会到中国和外星人之间的强烈对立。剧中还有许多令人振奋的瞬间，比如新中国发现外星人的弱点，并计划利用这一弱点击败外星人，以及新中国取得最终胜利的时刻。总的来说，《三体人不是无法战胜的》是一部可以激发现代人对爱国主义的战争精神的好戏，它的惊心动魄的战争场景更是令人称赞。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果

In [2]:
# 安装最新版本的 LangChain Python SDK（https://github.com/langchain-ai/langchain）
!pip install -U langchain

In [23]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

response_schemas = [
    ResponseSchema(name="role", description="role to answer the user's question, mentioned in the question"),
    ResponseSchema(name="content", description="content that the role used to answer the user's question")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介：
{format_instructions}"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["title", "era"],
    partial_variables={"format_instructions": format_instructions}
)

llm = OpenAI(temperature=.7, max_tokens=1500)


synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_parser=output_parser, output_key="synopsis", verbose=True)


In [24]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}中的'content'

来自《纽约时报》戏剧评论家对上述剧目的评价：
{format_instructions}"""

prompt_template = PromptTemplate(
    input_variables=["synopsis"],
    template=template,
    partial_variables={"format_instructions": format_instructions}
)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_parser=output_parser, output_key="review", verbose=True)

In [25]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [26]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"role": string  // role to answer the user's question, mentioned in the question
	"content": string  // content that the role used to answer the user's question
}
```

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{'role': '剧作家', 'content': '《三体人不是无法战胜的》是一部关于21世纪新中国的戏剧，讲述了一群勇敢的年轻人不畏惧威胁，勇于接受挑战，最终战胜了来自三体人的威胁的故事。他们通过坚强的意志力和合作精神，帮助拯救了整个世界。'}中的'content'

来自《纽约时报》戏剧评论家对上述剧目的评价：
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"role": string  // role to answer the user's question, mentioned

{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': {'role': '剧作家',
  'content': '《三体人不是无法战胜的》是一部关于21世纪新中国的戏剧，讲述了一群勇敢的年轻人不畏惧威胁，勇于接受挑战，最终战胜了来自三体人的威胁的故事。他们通过坚强的意志力和合作精神，帮助拯救了整个世界。'},
 'review': {'role': '戏剧评论家',
  'content': '《三体人不是无法战胜的》是一部极具想象力和冒险精神的戏剧，它展示了21世纪新中国的年轻人如何勇敢地面对挑战，凭借勇气和团结协作，最终成功拯救了整个世界。剧中讲述的故事充满了激情，演员们的表演也十分出色，令观众全身心投入。它既是一部社会主义的励志戏剧，也是一部深具感染力的艺术作品，值得大家一看。'}}